In [11]:
import os
import streamlit as st
import tensorflow as tf
from tensorflow.keras.models import load_model 
from PIL import Image
import numpy as np
import json

In [12]:
MODEL_PATH = r"D:/project 7th sem/minor project/model/plant_disease_prediction_model.h5"
CLASS_INDICES_PATH = r"D:/project 7th sem/minor project/Dataset/class_indices.json"


In [17]:
# Load the trained model
@st.cache_resource  # Caches the model to prevent reloading it every time
def load_prediction_model(model_path):
    try:
        if os.path.exists(model_path):
            return load_model(model_path, compile=False)  # Skip compilation
        else:
            st.error(f"Model file not found at {model_path}")
            return None
    except Exception as e:
        st.error(f"Error loading model: {str(e)}")
        return None

In [18]:
# Load class indices
@st.cache_data  # Caches the class indices for efficient reuse
def load_class_indices(class_indices_path):
    if os.path.exists(class_indices_path):
        with open(class_indices_path, 'r') as f:
            class_indices = json.load(f)
        # Reverse the dictionary to map indices to class names
        return {v: k for k, v in class_indices.items()}
    else:
        st.error(f"Class indices file not found at {class_indices_path}")
        return {}

2024-11-28 21:49:51.942 No runtime found, using MemoryCacheStorageManager


In [19]:
# Preprocess the uploaded image
def preprocess_image(image, target_size=(224, 224)):
    image = image.resize(target_size)  # Resize the image
    img_array = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

In [20]:
# Main Streamlit application
def main():
    st.title("Plant Disease Prediction App 🌱")
    st.write("Upload an image of a plant leaf to predict its disease.")

    # Load the model and class indices
    model = load_prediction_model(MODEL_PATH)
    class_indices = load_class_indices(CLASS_INDICES_PATH)

    if model is None or not class_indices:
        st.error("Failed to load the model or class indices. Please check the file paths.")
        return

    # File uploader
    uploaded_file = st.file_uploader("Choose a plant leaf image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        # Display the uploaded image
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Image", use_column_width=True)
        st.write("Processing...")

        # Preprocess and predict
        preprocessed_img = preprocess_image(image)
        predictions = model.predict(preprocessed_img)
        predicted_class_index = np.argmax(predictions, axis=1)[0]

        # Get the class name
        predicted_class_name = class_indices.get(predicted_class_index, "Unknown")

        # Display the result
        st.success(f"Predicted Disease: **{predicted_class_name}**")
        st.write(f"Prediction Confidence: {predictions[0][predicted_class_index] * 100:.2f}%")

if __name__ == "__main__":
    main()